<a href="https://colab.research.google.com/github/silcheon/UROP_project/blob/master/squat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
import numpy as np
import math
import glob
import utils

from parse import load_ps
from pprint import pprint
from scipy.signal import medfilt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import DistanceMetric
from sklearn.metrics import classification_report

In [ ]:
files = utils.files_in_order('poses_compressed/squat')

X_train_names, X_test_names = train_test_split(files[1:16], test_size=0.2, random_state=42)
y_train = utils.get_labels(X_train_names)#x_train_names의 목록
y_test = utils.get_labels(X_test_names)#x_test_names의 목록

pprint(X_train_names)
pprint(y_train)
pprint(X_test_names)
pprint(y_test)

In [ ]:
def load_features(names, flag):
    output1 = [] # List of upper arm torso angles
    output2 = [] # List of upper_leg_under_leg_angle
    output3 = [] # List of under_leg_ground_angle
    
    for filename in names:
        if flag == 0: # flag 0 이면 기존 training, test 
            ps = load_ps('poses_compressed/squat/'+filename)
        else: # 1 flag 이면 새로 만든 값
            ps = load_ps('numpy/'+filename)
        
        poses = ps.poses

        right_present = [1 for pose in poses 
                if pose.rankle.exists and pose.rknee.exists and pose.rhip.exists]
        left_present = [1 for pose in poses
                if pose.lankle.exists and pose.lknee.exists and pose.lhip.exists]
        right_count = sum(right_present)
        left_count = sum(left_present)
        side = 'right' if right_count > left_count else 'left'

        if side == 'right':
            joints = [(pose.rankle, pose.rknee, pose.rhip, pose.neck) for pose in poses]
        else:
            joints = [(pose.lankle, pose.lknee, pose.lhip,pose.neck) for pose in poses]

        # filter out data points where a part does not exist
        joints = [joint for joint in joints if all(part.exists for part in joint)]
        
        torso_vecs = np.array([(joint[3].x - joint[2].x, joint[3].y - joint[2].y) for joint in joints])#몸통벡터
        upper_leg_vecs = np.array([(joint[2].x - joint[1].x, joint[2].y - joint[1].y) for joint in joints])#허벅지 벡터
        under_leg_vecs = np.array([(joint[1].x - joint[0].x, joint[1].y - joint[0].y) for joint in joints])#정강이 벡터
        ground_vecs=np.array([((joint[0].x)+4,0) for joint in joints])#x축과 평행한 벡터(땅벡터)

        torso_vecs = torso_vecs / np.expand_dims(np.linalg.norm(torso_vecs, axis=1), axis=1)
        upper_leg_vecs = upper_leg_vecs / np.expand_dims(np.linalg.norm(upper_leg_vecs, axis=1), axis=1)
        under_leg_vecs = under_leg_vecs / np.expand_dims(np.linalg.norm(under_leg_vecs, axis=1), axis=1)
        ground_vecs = ground_vecs / np.expand_dims(np.linalg.norm(under_leg_vecs, axis=1), axis=1)

        upper_leg_torso_angle = np.degrees(np.arccos(np.clip(np.sum(np.multiply(upper_leg_vecs, torso_vecs), axis=1), -1.0, 1.0)))
        upper_arm_torso_angle_filtered = medfilt(medfilt(upper_leg_torso_angle, 5), 5)
        
        upper_leg_under_leg_angle = np.degrees(np.arccos(np.clip(np.sum(np.multiply(upper_leg_vecs, under_leg_vecs), axis=1), -1.0, 1.0)))
        upper_leg_under_leg_angle_filtered = medfilt(medfilt(upper_leg_under_leg_angle, 5), 5)

        under_leg_ground_angle = np.degrees(np.arccos(np.clip(np.sum(np.multiply(under_leg_vecs, ground_vecs), axis=1), -1.0, 1.0)))
        under_leg_ground_angle_filtered = medfilt(medfilt(under_leg_ground_angle, 5), 5)

        output1.append(upper_arm_torso_angle_filtered.tolist())
        output2.append(upper_leg_under_leg_angle_filtered.tolist())
        output3.append(under_leg_ground_angle_filtered.tolist())
     
        
    return output1, output2, output3

X_train_1, X_train_2, X_train_3 = load_features(X_train_names, 0)
X_test_1, X_test_2, X_train_3 = load_features(X_test_names, 0)
print(X_train_names)
print(len(X_train_1))

'''
X_train_1=load_features(X_train_names,0)
X_train_2=load_features(X_train_names,0)
X_test_1=load_features(X_test_names, 0)
X_test_2=load_features(X_test_names, 0)
X_new_names = ['test.npy']
X_new_1= load_features(X_new_names, 1)
X_new_2= load_features(X_new_names, 1)
'''

In [ ]:
def KNN(X_names, X_1, X_2):
    predictions = []
    
    for example in range(len(X_names)): # X_test_names
        # Store the average distance to good and bad training examples
        f1_good, f1_bad, f2_good, f2_bad = [[] for i in range(4)]

        # Compare distance of current test example with all training examples
        for i in range(len(X_train_1)):
            print(np.array(X_train_1[i]),np.array(X_1[example]))
            print(np.array(X_train_2[i]),np.array(X_2[example]))
            
            dist1 = utils.DTWDistance(np.array(X_train_1[i]),np.array(X_1[example]))
            dist2 = utils.DTWDistance(X_train_2[i], X_2[example])
            if y_train[i]:
                f1_good.append(dist1)
                f2_good.append(dist2)
            else:
                f1_bad.append(dist1)
                f2_bad.append(dist2)
        good_score = np.mean(f1_good) + np.mean(f2_good)
        bad_score = np.mean(f1_bad) + np.mean(f2_bad)

        if good_score < bad_score:
            predictions.append(1)
        else:
            predictions.append(0)
    #print(classification_report(y_test, predictions, target_names=['correct', 'incorrect']))
    print(predictions)

    
print(X_test_names)
KNN(X_test_names, X_test_1, X_test_2) # 기존 test data 
# KNN(X_new_names, X_new_1, X_new_2) # 영상에서 새로 추출한 데이터

In [ ]:
!python3 utils.py